# Lab 6 - Coder Agent

A coder agent doesn't necessarily mean that it generates code - but more broadly that it can write code and run it in order to solve a task.

In [1]:
from agents import Agent, Runner, trace, function_tool
import docker
import tempfile
import os

In [3]:
docker_client = docker.from_env()
image = "python:3.12-slim"

In [ ]:
docker_client.containers.run(image="python:3.12-slim",
                             command=["python", "-c", "print(2+2)"],
                             remove=True)

In [11]:
@function_tool
def execute_python(code: str) -> str:
    """
    Execute the given Python code inside a Docker container with python:3.12-slim,
    and return whatever is printed to stdout.
    You must print the result of the code to stdout in order to retrieve it.
    This uses the python:3.12-slim image and so it does not have scientific libraries installed;
    write simple python 3.12 code using the standard library only. Do not use numpy or scipy.

    Args:
        code: The Python code to run.

    """
    print(f"Executing code: {code}")
    with tempfile.TemporaryDirectory() as tmpdir:
        script_path = os.path.join(tmpdir, "script.py")
        with open(script_path, "w") as f:
            f.write(code)
        logs = docker_client.containers.run(
            image=image,
            command=["python", "/tmp/script.py"],
            volumes={tmpdir: {"bind": "/tmp", "mode": "ro"}},
            remove=True, 
            stdout=True,
            stderr=True,
        )
    result = logs.decode("utf-8")
    print(f"Result: {result}")
    return result

In [ ]:
execute_python.description

In [13]:
instructions = """
You are a Coder Agent that writes python, uses a tool to execute the code, and returns the results,
in order to solve the user's problem.
"""

input = """
Write a python function to calculate the price of an option using the Black-Scholes model.
Then use your tool to execute the code and calculate the price of this option:
GOOGL
Stock Price: 150
Strike Price: 155
Time to maturity: 3 years
Risk free rate: 1%
Volatility: 20%
Dividend Yield: 0%
Respond with the calculated price of the option only.
"""

In [ ]:

agent = Agent("Coder Agent", model="gpt-4.1-mini", instructions=instructions, tools=[execute_python])

with trace("Coder Agent"):
    result = await Runner.run(agent, input)
    print(result.final_output)